<a href="https://colab.research.google.com/github/h2llo/h2llo/blob/Thisismain/koBERT_%EB%8B%A4%EC%A4%91%EB%B6%84%EB%A5%98%EB%AA%A8%EB%8D%B8_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 들어가기 앞서
- Colab에서 CUDA Coding을 하기위한 명령어
    - [코드 셀]에서 command-line 명령어 실행시키기

    - !ls : 현재 디렉토리의 내용을 보여준다.
    - %cd dir: dir 디렉토리로 이동한다.
    - %pwd: 현재 위치한 디렉토리 위치를 보여준다.
    - !git: git 명령어를 실행시킨다.

- GPU 장치와 관련되어 사용되는 명령어

    - !nvidia-smi: 현재 사용하고 있는 GPU의 스펙과 작동 상황을 보여준다.
    - !nvcc: Nvidia CUDA Compiler를 실행시킨다.

## koBERT 다중분류모델
- koBERT를 이용한 모델엔 한국어 대화 문장을 7가지의 감정 (기쁜, 슬픔, 놀람, 분노, 공포, 혐오, 중립)으로 분류하는 모델을 학습시키는 것

- 코드는 KoBERT 깃허브 https://github.com/SKTBrain/KoBERT 에있는 네이버 영화평 이중분류 예시 코드 https://colab.research.google.com/github/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb#scrollTo=itIExnuLbSap 를바탕으로 작성했다.

- 사실 혼자선 도저히 이해가 되지않아 구글링을 한 결과 어떤 블로그 https://velog.io/@seolini43/KOBERT%EB%A1%9C-%EB%8B%A4%EC%A4%91-%EB%B6%84%EB%A5%98-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EA%B8%B0-%ED%8C%8C%EC%9D%B4%EC%8D%ACColab 의글을 바탕으로 코드를 익혔다.

- 학습데이터는 AIHUB에 오픈소스인 '한국어 감정  정보가 포함된 단발성 대화 데이터셋'을 다운받아서 사용하였다. 이 데이터는 SNS나 온라인 댓글에 대한 웹 크롤링을 실시한 3만여개의 문장으로 각 문장에 대해 7가지의 감정으로 레이블링이 되어있다. 모델을 7가지의 감정으로 분류한 것도 이때문이다.

In [ ]:
!nvidia-smi

In [ ]:
# 환경설정
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 1.8 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 13.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595722 sha256=5a405c7314dc2162f620bcfefd4b65ec386d17a60623d838c261a860a9e6f3c2
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 12.5 MB/s 
     |████████████████████████████████| 769 kB 11.0 MB/s 
     |████████████████████████████████| 3.0 MB 36.4 MB/s 
     |████████████████████████████████| 895 kB 48.2 MB/s 


In [ ]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-a2lq5r4a
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-a2lq5r4a
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=dc16dacf594b46e467094a0e67551d2a7387eba42d76e3fd1a645d06433d74f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-tqh4an13/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [ ]:
# 필요 라이브러리 import
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
#koBERT
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


## 1. 필요 데이터 import, 전처리
- 학습에 필요한 데이터셋은 aihub에서 가져왔고 3만8천여개의 sns 대화내용을 수집한 것




In [ ]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# data프레임의 형태로 지정
import pandas as pd
chatbot_data = pd.read_excel('/content/drive/MyDrive/projectColor/dataSet/한국어_단발성_대화_데이터셋.xlsx')

In [ ]:
# 무작위 컬럼 10개 가져와서 확인해보기
chatbot_data.sample(n=10)

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
24984,주거비용 개선 안하면 저출산은 해결 안된다..,중립,NaN,NaN,NaN,NaN,NaN
3613,제가 사회공포증이 좀 있어서 정신과를 가보려고 하는데요,공포,NaN,NaN,NaN,NaN,NaN
689,사태에 심각성도 느꼈지만...,공포,NaN,NaN,NaN,NaN,NaN
29611,자리에 올라 전세계 팬들에게 감동을 ㅠㅠ,행복,NaN,NaN,NaN,NaN,NaN
5057,이런생각이 드는건 뭘까요?,공포,NaN,NaN,NaN,NaN,NaN
37145,노인들 대상으로 전화 설문했겠지 설문조사는 믿을수가 없다.,혐오,NaN,NaN,NaN,NaN,NaN
6732,진짜 아바타야대박...,놀람,NaN,NaN,NaN,NaN,NaN
14345,미리 불구속 짜놓고 고민하는 척!,분노,NaN,NaN,NaN,NaN,NaN
26878,이왕 만드는거 피자를 만들면서 가는게 더 낫지 않아?,중립,NaN,NaN,NaN,NaN,NaN
23988,기아에서 백용환 댈구오면 된다,중립,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Sentence엔 대화 내용이 적혀있고 Emotion엔 어떤 감정인지 레이블링 되어있음
# NaN이라 적혀있는 컬럼은 필요없으니 제거하고 7가지 감정을 숫자로 처리
chatbot_data.loc[(chatbot_data['Emotion'] == "공포"), 'Emotion'] = 0  #공포 => 0
chatbot_data.loc[(chatbot_data['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
chatbot_data.loc[(chatbot_data['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
chatbot_data.loc[(chatbot_data['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
chatbot_data.loc[(chatbot_data['Emotion'] == "중립"), 'Emotion'] = 4  #중립 => 4
chatbot_data.loc[(chatbot_data['Emotion'] == "행복"), 'Emotion'] = 5  #행복 => 5
chatbot_data.loc[(chatbot_data['Emotion'] == "혐오"), 'Emotion'] = 6  #혐오 => 6

data_list = []
for q, label in zip(chatbot_data['Sentence'], chatbot_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
# 이렇게 하면 문장과 숫자로 입력된 값이 'data_list'에 저장되는데 그중 몇개를 출력해서 확인해본다
print(data_list[0])
print(data_list[1000])
print(data_list[5000])
print(data_list[10000])
print(data_list[15000])
print(data_list[30000])
print(data_list[-1])

['언니 동생으로 부르는게 맞는 일인가요..??', '0']
['나보다 더 큰거 가터ㅋㅋㅋ 뇌랑 얼글이랑 손두ㅋㅋ', '0']
['그날 제가 일정이 있어서 컴퓨터를 못할 것 같아요...', '0']
['외국 어학연수 갔다온 여자들은 문란하다는 말이 사실?', '1']
['18년은 18년동안 빵에가두고를 18번을 시켜도국민의 원성이 잠들수 있을까요?', '2']
['  요리하는것이 숙제하는것처럼 힘든저에게 용기나게 해주시고 할수 있을것같은 희망을 주셔서감사합니다!!', '5']
['와이프도 그렇고 댓글 다 볼텐데 이휘재 좀 하차 하라고 전해주세요', '6']


## 2.trainData와 testData로 분류하기
- 이제 trainData와 testData로 분류하여 모델에 학습시키면 된다.
- split비율은 7:3 국룰을 따르려 했으나 train데이터가 적어질것을 우려해 4:1로 나눈다.

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

28945
9649


## 3. koBERT 입력 데이터로 만들기
- 각 데이터가 koBERT 모델의 입력으로 들어갈 수 있는 형태가 되도록 토큰화, 정수 인코딩, 패딩 등을 해주어야 함.
- 예시코드에 입력데이터의 형태로 만들어주는 클래스가 있어서 똑같이 작성했다.

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

# - import한 BERTSentenceTransform이라는 모듈을 사용하는것을 알 수 있음
# - 알고리즘은 잘 모르겠으나 이 모듈을 통해 토큰화와 패딩이 이뤄지는것을 알 수있음


In [ ]:
# parameters 세팅
# - 파라미터값은 정답이 없고 가장 좋은값을 찾으려 쌩고생을 해야하기 때문에 일단 지금은
#   예시코드에 있는거 그대로 설정해준다.
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
# 버트 토크나이저와 위에 정의한 클래스를 적용해 토큰화와 패딩을 해준다.
tokenizer = get_tokenizer()
token = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

X_train = BERTDataset(dataset_train, 0, 1, token, max_len, True, False)
X_test = BERTDataset(dataset_test, 0, 1, token, max_len, True, False)

using cached model


In [ ]:
# 잘 되어있는지 확인해본다
print(X_train[0])
print(X_test[0])

(array([   2, 1189,  517, 6188, 7245, 7063,  517,  463, 3486, 7836, 5966,
       1698,  517, 6188, 7245, 7063,  517,  463, 1281, 7870, 1801, 6885,
       7088, 5966, 1698, 5837, 5837,  517, 6188, 7245, 6398, 6037, 7063,
        517,  463,  517,  463,  517,  364,  517,  364,    3,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32), array(42, dtype=int32), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32), 5)
(array([   2,  520, 3220, 7096, 6882, 3036, 7408,  520,  517, 7102, 3095,
       5330, 3396, 5561, 7782, 5837, 5512, 6999,  517,   54,    3,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
    

In [ ]:
# 총 3개의 array가 출력되는데, 첫번째는 패딩된 시퀀스, 두 번째는 길이와 타입에 대한 내용, 세번째는 어텐션 마스크 시퀀스이다.
# ★★★중요 : BERT에 데이터가 입력되었을 때 어텐켠 함수가 적용되어 연산이 되는데
#              1로 패딩된 값들은 연산할 필요가 없기 때문에 연산을 하지 않아도 된다는 데이터가 있어야
#              하는데 그게 어텐션 마스크 시퀀스라 한다.
#              따라서 BERT나 koBERT에는 어텐션마스크 데이터도 항상 같이 입력되어야 한다.

In [ ]:
# torch형식의 dataSet을 만들어줌.
train_dataloader = torch.utils.data.DataLoader(X_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(X_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## 4.koBERT 학습모델 만들기
- 학습시킬 koBERT 모델을 만들어야 하는데 아래 코드에서 다중분류할 클래스 수 만큼 num_classes 변수를 설정해줘야 함.
- 7가지의 class를 분류하기 때문에 7로 입력

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# 뭔지는 모르겠지만 예시코드에 하라고 했으니 코드를 실행시킨다.
# BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

# optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

## 5. koBERT 모델 학습시키기
- 코드를 실행하여 koBERT 모델을 학습시켜 준다.

In [ ]:
# 아마 만들어 놓은 koBERT모델에 학습시키는 코드인듯 싶다.
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/453 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.110454559326172 train acc 0.078125
epoch 1 batch id 201 loss 1.2695311307907104 train acc 0.2761971393034826
epoch 1 batch id 401 loss 1.2274527549743652 train acc 0.3854036783042394
epoch 1 train acc 0.4000028405401896


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 2 batch id 1 loss 1.1605730056762695 train acc 0.59375
epoch 2 batch id 201 loss 1.0474213361740112 train acc 0.5361473880597015
epoch 2 batch id 401 loss 1.001801609992981 train acc 0.5692019950124688
epoch 2 train acc 0.5739920140241527


  0%|          | 0/151 [00:00<?, ?it/s]

epoch 2 test acc 0.5445626098121368


  0%|          | 0/453 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.8815680146217346 train acc 0.65625


## 6. 새로운 문장 테스트
- 새로운 문장을 테스트 할때엔 입력되는 문장을 koBERT의 입력 형식으로 바꿔주는 코드를 작성해주어야 한다.
- 아래 코드를 작성하여 토큰화, 패딩, tensor를 바꿔주고 예측하는 'predict' 함수를 만들어 준다.

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

#패딩, tensor를 바꿔주는 함수 작성부분
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)


        test_eval=[]
        # 메소드 실행 후 결과값에 따라 7가지 감정을 출력해주는 부분
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [ ]:
# 테스트
# 0을 입력하면 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")